# Automating the transcription and summarization of recorded speech using OpenAI's Whisper and chatGPT 

### Requirements (using a Python 3.8 conda environment)

In [ ]:
!pip install git+https://github.com/openai/whisper.git -q
!pip install spacy
!pip install pyaudio
#!pip install tiktoken
!pip install nltk
!python -m spacy download en_core_web_sm

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import openai
import pyaudio
import whisper
import time
import wave # should come with Python s
# Apply the API key
openai.api_key = "sk-xxxxxxxx....??????"

### Transcription of an online recording

The code here shown extends from the example shown in [betterprogramming](https://betterprogramming.pub/building-a-thought-summarization-app-with-whisper-and-gpt-3-90c2d8653faa).

In [4]:
#KMP_DUPLICATE_LIB_OK=TRUE()
CHANNELS = 1
FORMAT = pyaudio.paInt32
RATE = 44100
CHUNK = 1024
# Record for 20 seconds
time_recording = 20

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,channels=CHANNELS,rate=RATE,input=True,frames_per_buffer=CHUNK)
print("Start recording")
frames = []

seconds = 10
for i in range(0, int(RATE / CHUNK * seconds)):
    data = stream.read(CHUNK)
    frames.append(data)
    

print("Recording stopped")
stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open("output.wav", "wb")
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()


write_text_file = True
audio_file_path = "./output.wav"

# Load the whisper model
model = whisper.load_model("base.en")
#result = model.transcribe(audio_file_path)
result = model.transcribe(audio_file_path, fp16=False, language='English')
if write_text_file==True:    
    with open("rec_wav.txt", "w+") as f:
        f.write(result["text"])

print(result["text"])

Start recording
Recording stopped
 Thank you.


### Transcription of an offline recording

In [ ]:
write_text_file = True

audio_file_path = "./Albert Camus Straße 38.m4a"

# Load the whisper model
model = whisper.load_model("base.en")
#result = model.transcribe(audio_file_path)
result = model.transcribe(audio_file_path, fp16=False, language='English')
if write_text_file==True:    
    with open("rec_m4a.txt", "w+") as f:
        f.write(result["text"])

print(result["text"])

### Summarization

In [ ]:
import openai

with open("rec_wav.txt") as f:
    contents = f.readlines()
print(contents[0])

# Define the prompt for GPT-3
text = contents #result["text"]
prompt = (f"summarize this text using bulletpoints: {text}")

# Get a response from GPT-3
response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    max_tokens=1024,
    n =1,
    stop=None,
    temperature=0.5
)

# Get the summary from the response
summary = response.choices[0].text

# Print the summary
print(summary)

### Summarization by chunks

In [21]:
import openai
import spacy
from spacy.lang.en import English
nlp = spacy.load("en_core_web_sm")

fname = "rec_m4a.txt"  # I replaced the recorded speech for a text lumped from the Wiki on the Chilean civil war of 1981 8-)
#fname = "idea_2.txt"

def text_to_chunks(text):
    chunks = [[]]
    chunk_total_words = 0

    sentences = nlp(text)

    for sentence in sentences.sents:
        chunk_total_words += len(sentence.text.split(" "))

        if chunk_total_words > 500:
            chunks.append([])
            chunk_total_words = len(sentence.text.split(" "))

        chunks[len(chunks)-1].append(sentence.text)

    return chunks

def summarize_text(text):
    #prompt = f"Summarize the following text in 3 sentences:\n{text}"
    prompt = (f"summarize this text: {text}")

    response = openai.Completion.create(
      engine="text-davinci-003", 
      prompt=prompt,
      temperature=0.5, 
      max_tokens=1049, 
      top_p=1, 
      frequency_penalty=0,
      presence_penalty=1
    )

    return response["choices"][0]["text"]


with open(fname) as f:
    contents = f.readlines()
#print(contents[0])

text = contents[0] #result["text"]

chunks = text_to_chunks(text)

chunk_summaries = []

for idx, chunk in enumerate(chunks):
    chunk_summary = summarize_text(" ".join(chunk))
    print("Summary of chunk:", idx+1)
    chunk_summaries.append(chunk_summary)

summary = " ".join(chunk_summaries)

# Print the summary
print("Appended summaries : ", summary)


Summary of chunk: 1
Summary of chunk: 2
Summary of chunk: 3
Summary of chunk: 4
Summary of chunk: 5
Summary of chunk: 6
Appended summaries :  

The Chilean Civil War of 1891 was a conflict between the Chilean Army and Navy, with the latter siding with Congress and the former supporting President José Manuel Balmaceda. The war ended with the defeat of the president's forces and Balmaceda committing suicide, marking the end of the Liberal Republic and the beginning of the Parliamentary Era. It began as a disagreement between Balmaceda and Congress over the power to appoint ministers, and escalated when Balmaceda attempted to nominate a close friend as his successor. Congress refused to approve a budget and Balmaceda declared an official budget for 1891, which was interpreted by the opposition as illegal. 

In August 1891, Vice-President Waldo Silva and President of the Chamber of Deputies Ramón Barros Luco issued a proclamation appointing Captain Jorge Montt commander of the navy in prot